In [ ]:
# Importation de modules

import unicodedata as uni
import re
import os
import csv as csvlib
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import xml.etree.ElementTree as ET
ns = "http://www.tei-c.org/ns/1.0"
from itertools import combinations
from tqdm.auto import tqdm
from math import factorial
from numpy import array, mean, percentile
import pandas as pd


In [ ]:
# Fonction pour supprimer les diacritiques

def strip_accents(string):
   return ''.join(c for c in uni.normalize('NFD', string)
                  if uni.category(c) != 'Mn')

In [ ]:
# Variables générales

# définir la fourchette de caractères des phrases que l'on va comparer
min_phr_len = 20
max_phr_len = 2000

# définir la distance de levenshtein minimale pour créer un lien
min_lev_distance = 0.80

In [ ]:
# Extraire les phrases et créer les squelettes textuels

# initier un compteur de phrases pour assigner des identifiants dans les noeuds
id_phrase = 0
# créer un dictionnaire nodes[id_text + "ph" + id_phrase] = [id_text, id_phrase, text, label]
nodes = {}
# créer un dictionnaire edges[source] = [target, edge_type, lev_distance]
edges = {}

for root_path, _, files in os.walk("mazarinades"):
    for name in files:
        if(not name.endswith(".xml")):
            continue
        # pour chaque fichier .xml du dossier mazarinades
        with open(os.path.join(root_path, name), 'r', encoding='utf-8') as f:
            xml = f.read()
            root = ET.fromstring(xml)
            id_phrase = 0

            # récupérer le titre du document pour l'afficher en label des noeuds
            title = root.find(".//{" + ns + "}title").text
            try:
                label = re.sub(r'\s+', ' ', title).replace('\n', '')
            except:
                label = "no_title"

            # récupérer l'identifiant dans id_text
            id_text = name.replace(".xml", "")
            # insérer le noeud titre dans le dictionnaire de noeuds
            nodes[id_text + "ph" +
                  str(id_phrase)] = [id_text, str(id_phrase), "null", label]

            # extraire le texte et supprimer les balises, les espaces multiples, les tirets, les diacritiques et mettre tout en minuscules
            full_text = re.sub(
                r'\s+', ' ', (re.sub(r'<.+?>|<\/?|\/?>', ' ', xml.split("body>")[1])))
            full_text = strip_accents(full_text.replace("¬", "").lower())

            # créer un noeud pour chaque phrase qui dépasse 20 caractères
            phrases = re.findall("[^\.\?\!]{" + str(min_phr_len) + ",}", full_text)
            for phrase in phrases:
                id_phrase += 1
                nodes[id_text + "ph" + str(id_phrase)] = [id_text,
                                                     id_phrase, phrase, "null"]
                # créer un lien avec la phrase précédente
                edges[id_text + "ph" +
                      str(id_phrase - 1)] = [id_text + "ph" + str(id_phrase), "precedence", "null"]

# vérifier le nombre de phrases
print(len(nodes))
                    

In [ ]:
# Répartir les phrases dans des groupes

# créer une liste de graduations pour les groupes de taille de phrase
graduations = []

# définir des graduations en fonction de la taille minimale, maximale et de l'écart de levenshtein minimal
i = min_phr_len
while i < max_phr_len:
    graduations.append(round(i))
    i = i/min_lev_distance
graduations.append(max_phr_len)

# répartir les noeuds dans des groupes qui couvrent 3 graduations
groups = [[(id, node) for id, node in nodes.items()
           if graduations[i] < len(node[2]) < graduations[i+2]]
          for i in range(len(graduations)-2)]

# attribuer à chaque phrase un indice entre 0 et 1, qui indique sa position entre les 2 bornes du groupe
# ne comparer qu'avec les phrases qui ont un indice entre 0.25 plus grand et 0.25 plus petit


In [ ]:
# Créer les liens de proximité entre phrases

# ajouter une barre de progression
total_comb = sum(
    [factorial(len(group)) // (2*factorial(len(group) - 2)) for group in groups])
with tqdm(total=total_comb) as pbar:
    # comparer les phrases de chaque groupe entre elles
    for group in groups:
        for (id1, node1), (id2, node2) in combinations(group, 2):
            # si les phrases ne font pas partie du même texte
            if node1[0] != node2[0]:
                lev_distance = fuzz.ratio(node1[2], node2[2]) / 100
                # créer des liens avec les phrases qui sont proches à 80%
                if (lev_distance >= min_lev_distance):
                    edges[id1] = [id2, "proximity", lev_distance]
            pbar.update(1)
